# Lentz Soliton Warp Drive

This notebook explores the Lentz soliton warp drive.

**Reference**: Lentz, E. W. (2021). Class. Quantum Grav. 38, 075015.

**[ASSUMPTION]**: This implementation is an interpretation of Lentz's paper. The specific parameterizations may differ from the original work.

## Key Innovation

Lentz proposed using **soliton-like profiles** with **electromagnetic + plasma field sources** to create a warp drive configuration that could potentially satisfy energy conditions through matter field contributions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from warpbubblesim.metrics import AlcubierreMetric, LentzMetric
from warpbubblesim.gr import compute_energy_density, check_energy_conditions
from warpbubblesim.gr.geodesics import integrate_geodesic
from warpbubblesim.viz.fields2d import plot_energy_density, plot_field_2d

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. The Soliton Profile

Lentz uses a hyperbolic secant (sech) profile instead of the tanh function:

$$f(r) \sim \text{sech}^2\left(\frac{r}{R}\right)$$

This gives a soliton-like shape that naturally appears in many physical systems.

In [ ]:
# Create Lentz metric
lentz = LentzMetric(
    v0=0.5,
    R=1.0,
    sigma=5.0,
    amplitude=0.1,
    hyperbolic_param=0.5
)

info = lentz.info()
print(f"Metric: {info['name']}")
print(f"Citation: {info['citation']}")
print(f"Note: {info.get('note', 'N/A')}")
print(f"Parameters: {info['parameters']}")

## 2. Shape Function Comparison

In [ ]:
# Compare soliton vs tanh profiles
r = np.linspace(-3, 3, 200)

alc = AlcubierreMetric(v0=1.0, R=1.0, sigma=8.0)
lentz = LentzMetric(v0=1.0, R=1.0, sigma=5.0)

plt.figure(figsize=(10, 6))

# Alcubierre tanh profile
f_alc = [alc.shape_function(abs(ri)) for ri in r]
plt.plot(r, f_alc, 'b-', linewidth=2, label='Alcubierre (tanh)')

# Lentz soliton profile
f_lentz = [lentz.soliton_profile(abs(ri)) for ri in r]
plt.plot(r, f_lentz, 'r-', linewidth=2, label='Lentz (sech²)')

# Pure sech² for comparison
sech2 = 1.0 / np.cosh(r)**2
plt.plot(r, sech2, 'g--', linewidth=1, alpha=0.7, label='sech²(r)')

plt.xlabel('r')
plt.ylabel('f(r)')
plt.title('Shape Function Comparison: Tanh vs Soliton')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Energy Density Distribution

In [ ]:
# Compare energy density
alc = AlcubierreMetric(v0=0.5)
lentz = LentzMetric(v0=0.5)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_energy_density(alc, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[0])
axes[0].set_title('Alcubierre Energy Density')

plot_energy_density(lentz, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[1])
axes[1].set_title('Lentz Soliton Energy Density')

plt.tight_layout()
plt.show()

## 4. Einstein-Maxwell-Plasma Theory

Lentz's key idea is to source the metric with electromagnetic and plasma fields:

$$G_{\mu\nu} = 8\pi (T^{\text{EM}}_{\mu\nu} + T^{\text{plasma}}_{\mu\nu})$$

The combined stress-energy can potentially satisfy energy conditions even when the pure geometry would violate them.

In [ ]:
# Estimate required EM field strengths
lentz = LentzMetric(v0=0.5)

# Sample points
test_points = [
    ('Center', [0, 0, 0, 0]),
    ('Wall region', [0, 1.0, 0.3, 0]),
    ('Edge', [0, 2.0, 0.5, 0]),
]

print("Estimated EM Field Requirements:")
print("="*50)

for name, coords in test_points:
    t, x, y, z = coords
    em_info = lentz.em_field_estimate(t, x, y, z)
    
    print(f"\n{name} ({x}, {y}, {z}):")
    print(f"  B field estimate: {em_info['magnetic_field_estimate']:.2e} (geometric units)")
    print(f"  E field estimate: {em_info['electric_field_estimate']:.2e} (geometric units)")

## 5. Parameter Study

In [ ]:
# Effect of amplitude parameter
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

amplitudes = [0.05, 0.1, 0.2, 0.5]

for ax, amp in zip(axes, amplitudes):
    metric = LentzMetric(v0=0.5, amplitude=amp)
    plot_energy_density(metric, x_range=(-3, 3), y_range=(-3, 3), 
                        nx=60, ny=60, ax=ax)
    ax.set_title(f'Amplitude = {amp}')

plt.suptitle('Lentz Energy Density vs Amplitude', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Energy Condition Analysis

In [ ]:
lentz = LentzMetric(v0=0.5, amplitude=0.1)
metric_func = lentz.get_metric_func()

# Check conditions at various points
test_points = [
    ('Center', [0, 0, 0, 0]),
    ('Near center', [0, 0.5, 0.2, 0]),
    ('Wall', [0, 1.0, 0.3, 0]),
    ('Outer region', [0, 2.0, 0.5, 0]),
    ('Far', [0, 4.0, 0, 0]),
]

print("Energy Conditions (geometric contribution only):")
print("="*70)
print("Note: Lentz claims EM+plasma sources can satisfy conditions")
print("-"*70)

for name, coords in test_points:
    coords = np.array(coords, dtype=float)
    conditions = check_energy_conditions(metric_func, coords, n_samples=20)
    
    print(f"\n{name}:")
    for cond_name, (satisfied, value) in conditions.items():
        status = "OK" if satisfied else "VIOLATED"
        print(f"  {cond_name}: {status:8s} ({value:+.4e})")

## 7. Field Sourcing Analysis

Lentz proposes that the required stress-energy can be provided by electromagnetic and plasma fields:

In [ ]:
from warpbubblesim.metrics.lentz import LentzFieldSourcing

lentz = LentzMetric(v0=0.5)
sourcing = LentzFieldSourcing(lentz)

# Analyze field sourcing requirements
analysis = sourcing.energy_condition_analysis()

print("Lentz Field Sourcing Analysis:")
print("="*50)
print(f"Min energy density: {analysis['min_energy_density']:.4e}")
print(f"Max energy density: {analysis['max_energy_density']:.4e}")
print(f"All positive (with EM+plasma): {analysis['all_positive']}")
print(f"\nNote: This is a simplified analysis; full treatment")
print(f"requires solving coupled Einstein-Maxwell-plasma equations")

## 8. Geodesics

In [ ]:
lentz = LentzMetric(v0=0.5)
metric_func = lentz.get_metric_func()

# Integrate geodesics
results = []
for x0 in np.linspace(-4, 4, 9):
    initial_coords = np.array([0.0, x0, 0.0, 0.0])
    initial_velocity = np.array([1.0, 0.0, 0.0, 0.0])
    
    result = integrate_geodesic(
        metric_func, initial_coords, initial_velocity,
        lambda_span=(0, 15), max_step=0.1
    )
    results.append(result)

plt.figure(figsize=(12, 8))

for result in results:
    coords = result['coords']
    plt.plot(coords[:, 1], coords[:, 0], alpha=0.7)

# Soliton trajectory
t_range = np.linspace(0, 15, 100)
x_soliton = [lentz.bubble_center(t) for t in t_range]
plt.plot(x_soliton, t_range, 'k--', linewidth=2, label=f'Soliton (v={lentz.v0}c)')

plt.xlabel('x')
plt.ylabel('t')
plt.title('Geodesics in Lentz Soliton Spacetime')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 9. Comparison with Other Positive-Energy Approaches

In [ ]:
from warpbubblesim.metrics import BobrickMartireMetric

# Compare Lentz and Bobrick-Martire approaches
lentz = LentzMetric(v0=0.3)
bm = BobrickMartireMetric(v0=0.3, positive_energy=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_energy_density(lentz, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[0])
axes[0].set_title('Lentz (EM+plasma sourcing)')

plot_energy_density(bm, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[1])
axes[1].set_title('Bobrick-Martire (subluminal)')

plt.suptitle('Positive-Energy Warp Drive Approaches (v=0.3c)', fontsize=14)
plt.tight_layout()
plt.show()

## 10. Physical Interpretation

Lentz's approach differs from others in several ways:

1. **Soliton-like profile**: More natural physical structure
2. **EM+plasma sourcing**: Uses known physics (electromagnetic fields)
3. **Coupled equations**: Requires solving Einstein-Maxwell-plasma system
4. **Potential positive energy**: Claims energy conditions can be satisfied

In [ ]:
print("Lentz Warp Drive Summary:")
print("="*60)
print()
print("APPROACH:")
print("  - Soliton-like metric profile (sech² function)")
print("  - Source with electromagnetic + plasma fields")
print("  - Solve coupled Einstein-Maxwell-plasma equations")
print()
print("CLAIMED ADVANTAGES:")
print("  - Positive energy density possible")
print("  - Uses known physics (EM fields)")
print("  - Natural soliton structure")
print()
print("CHALLENGES:")
print("  - Requires extremely strong EM fields")
print("  - Plasma configuration must be precisely maintained")
print("  - Stability questions remain")
print()
print("[ASSUMPTION]: Implementation is interpretation of published work")

## Summary

The Lentz soliton warp drive:

1. **[ASSUMPTION]** Implementation interprets the published paper
2. **Soliton profile** provides natural physical structure
3. **EM+plasma sourcing** attempts to satisfy energy conditions
4. **Requires extreme fields** - engineering challenges remain
5. **Novel approach** - combines warp drives with electromagnetic theory
6. **Open questions** about stability and practical implementation

Lentz's work represents an innovative approach to the warp drive problem, attempting to source the required stress-energy with known physics rather than exotic matter.